In [31]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tweepy
from config import *

In [32]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [23]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [7]:
# Twitter handles

# @BBCNorthAmerica
# @CBSNews
# @CNN
# @FoxNews
# @nytimes

In [42]:
# Define function to obtain tweets

def tweeter(handle):

    all_tweets = []
    
    for x in range(2):
        public_tweets = api.user_timeline(handle, page=x)
        all_tweets.append(public_tweets)
        tweet_list = [item for sublist in all_tweets for item in sublist]
    return tweet_list;

In [9]:
# Define function to analyze tweets

def vader(tweets):
    compound_list = []
    
    for text in tweets:

In [43]:
target = '@CNN'
with open('test_data.json', 'w') as outfile:  
    json.dump(tweeter(target), outfile)
print('Done!')

Done!


In [44]:
status = api.rate_limit_status()
#print(json.dumps(status, sort_keys=True, indent=4))
print(status['resources']['statuses']['/statuses/user_timeline']['remaining'])

898
